In [1]:
from subprocess import Popen, PIPE, STDOUT
from random import randint
import binascii
import os
from queue import Queue, Empty
from threading  import Thread
from time import sleep
import random
import nltk
nltk.download('averaged_perceptron_tagger')
from itertools import permutations, combinations
import spacy
from spacy.matcher import Matcher
from spacy.attrs import POS
nlp = spacy.load('en_core_web_sm')
import pandas as pd
nltk.download('punkt')
import numpy as np
import gensim
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from nltk.tokenize import sent_tokenize, word_tokenize
import timeit
from keras.models import Model
from keras.layers import *   
from progressbar import * 
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\sparr\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sparr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Using TensorFlow backend.


In [2]:
basic_actions = [
           'go north',
           'go south',
           'go west',
           'go east',
           'go northeast',
           'go northwest',
           'go southeast',
           'go southwest',
           'd',
           'u'
]

directions = ['north', 'south', 'east', 'west', 'northwest', 'northeast', 'southwest', 'southeast']

action_space = set()

for action in basic_actions:
    action_space.add(action)

command1_actions = ['open OBJ',
 'get OBJ',
 'set OBJ',
 'hit OBJ',
 'eat OBJ',
 'put OBJ',
 'cut OBJ',
 'dig OBJ',
 'ask OBJ',
 'fix OBJ',
 'give OBJ',
 'make OBJ',
 'wear OBJ',
 'move OBJ',
 'kick OBJ',
 'kill OBJ',
 'find OBJ',
 'play OBJ',
 'feel OBJ',
 'hide OBJ',
 'read OBJ',
 'fill OBJ',
 'go OBJ',
 'flip OBJ',
 'burn OBJ',
 'pick OBJ',
 'pour OBJ',
 'pull OBJ',
 'apply OBJ',
 'leave OBJ',
 'ask OBJ',
 'break OBJ',
 'enter OBJ',
 'curse OBJ',
 'shake OBJ',
 'carry OBJ',
 'spray OBJ',
 'burn OBJ',
 'inflate OBJ',
 'brandish OBJ',
 'donate OBJ',
 'squeeze OBJ',
 'attach OBJ',
 'find OBJ',
 'unlock OBJ',
 'banish OBJ',
 'read OBJ',
 'wait OBJ',
 'enchant OBJ',
 'feel OBJ',
 'pour OBJ']
                  

command2_actions = ['pour OBJ on DCT',
 'hide OBJ in DCT',
 'pour OBJ in DCT',
 'move OBJ in DCT',
 'hide OBJ on DCT',
 'flip OBJ for DCT',
 'fix OBJ with DCT',
 'spray OBJ on DCT',
 'dig OBJ with DCT',
 'cut OBJ with DCT',
 'pick OBJ with DCT',
 'squeeze OBJ on DCT',
 'pour OBJ from DCT',
 'fill OBJ with DCT',
 'brandish OBJ at DCT',
 'burn OBJ with DCT',
 'flip OBJ with DCT',
 'read OBJ with DCT',
 'hide OBJ under DCT',
 'carry OBJ from DCT',
 'inflate OBJ with DCT']
                    

filtered_tokens = ['Score', 'Moves']

In [3]:
filtered_tokens

['Score', 'Moves']

In [4]:
def enqueue_output(out, queue):
    for line in iter(out.readline, b''):
        queue.put(line)
    out.close()

In [5]:
def start_game():
    score = 0
    moves = 0
    p = Popen(['dfrotz.exe','zork1.z5'], stdin=PIPE, stdout=PIPE, stderr=PIPE, shell=True)
    q = Queue()
    t = Thread(target=enqueue_output, args=(p.stdout, q))
    t.daemon = True # thread dies with the program
    t.start()
    sleep(0.1)
    return(p, q, t, score, moves)

In [6]:
# read line without blocking
def readLine(q):
    cont = True
    narrative = ""
    while cont:
        try:  line = q.get_nowait() # or q.get(timeout=.1)
        except Empty:
            cont = False
        else: 
            narrative = narrative + line.decode("utf-8").replace('\n', " ").replace('\r', " ")
    if ('840726' in narrative): ## Opening narrative
        narrative = narrative[narrative.index('840726') + len('840726'):]
    try:
        score, moves = grab_score_moves(narrative)
        narrative = narrative[narrative.index('Moves: ')+len('Moves:')+3:-1].strip()
    except:  ## not valid move
        pass
    sleep(0.1)
    return(narrative, score, moves)

In [7]:
def look_surroundings(p):
    perform_action('look', p)

In [8]:
def check_inventory(p):
    perform_action('inventory', p)

In [9]:
def grab_score_moves(narrative):
    try:
        score = int(narrative[narrative.index('Score: ') + len('Score: '):][0:3].strip())
        moves = int(narrative[narrative.index('Moves: ') + len('Moves: '):][0:3].strip())
    except:  ## not valid move
        score = 0
        moves = 0
    return(score, moves)

In [10]:
def preprocess(text): 
    text = nltk.word_tokenize(text)
    processed = nltk.pos_tag(text)
    return processed

In [11]:
def detect_nouns(narrative):  #grabs nouns and adjective noun pairs from narrative
    
    processed = preprocess(narrative)
    pattern = 'NP: {<JJ>*<NN>}'
    cp = nltk.RegexpParser(pattern)
    cs = cp.parse(processed)
    noun_list =  [noun.text for noun in nlp(narrative).noun_chunks]
    for direction in directions:
        if direction in noun_list:
            noun_list.remove(direction)
    return(noun_list)
    
    # function to test if something is a noun
    #is_noun = lambda pos: pos[:2] == 'NN'
    # do the nlp stuff
    #tokenized = nltk.word_tokenize(narrative)
    #nouns = [word.lower() for (word, pos) in nltk.pos_tag(tokenized) if is_noun(pos)]
    #return(nouns)

In [12]:
def get_nouns(narrative):
    matcher = Matcher(nlp.vocab)
    matcher.add('Noun phrase', None, [{POS: 'NOUN'}])
    doc = nlp(narrative)
    matches = matcher(doc)
    noun_list = [doc[start:end].text for id, start, end in matches]
    for direction in directions:
        if direction in noun_list:
            noun_list.remove(direction)
    return(noun_list)

In [13]:
def generate_action_tuples(nouns):
    possible_actions = []
    similarities = []
    for i in basic_actions:
        possible_actions.append(i)
        similarities.append(.5)
    for i in nouns:
        for action1 in command1_actions:   ## first loop replaces 'x' in each action in command1_actions
            action_to_add = action1.replace('OBJ', i)
            possible_actions.append(action_to_add)
            try:
                similarities.append(model.similarity(word_tokenize(action_to_add)[0], i))
            except:
                similarities.append(0.02)
        noun_permutations = list(permutations(nouns, 2))    ## second loop replaces 'x' and 'y' in each action in command2_actions
        for action2 in command2_actions:
            for perm in noun_permutations:
                if (perm[0] == perm[1]):  ## ignore same noun acting on itself
                    pass
                else:
                    action_to_add = action2.replace('OBJ', perm[0])
                    action_to_add = action_to_add.replace('DCT', perm[1])
                    possible_actions.append(action_to_add)
                    try:
                        similarities.append(model.similarity(word_tokenize(action_to_add)[0], i))
                    except:
                        similarities.append(0.02)

    return possible_actions

In [14]:
from numpy.random import choice
def selectOne(action_space, similarities):
    return action_space[choice(len(action_space), p=similarities)]

In [28]:
import math
def add_to_action_space(action_space, actions):
    similarities = []
    
    for action in actions:
        action_space.add(action)
    for action in action_space:
        words = word_tokenize(action)
        verb = words[0]
        if verb in basic_actions:
            similarities.append(0.5)
        elif len(words)<3:
            noun = word_tokenize(action)[1]
            try:
                sim_score = model.similarity(verb, noun)**4
                if sim_score < 0:
                    sim_score = 0.05**4
                similarities.append(sim_score)
            except:
                similarities.append(0.02**4)
                
        else: 
            try:
                noun1 = word_tokenize(action)[1]
                prep = word_tokenize(action)[2]
                noun2 = word_tokenize(action)[3]
                sim_score1 = model.similarity(verb, noun1)
                sim_score2 = model.similarity(prep, noun2)
                sim_score = ((sim_score1 + sim_score2)/2)**4
                if sim_score < 0:
                    sim_score = 0.05
                similarities.append(sim_score**4)
            except:
                similarities.append(0.02**4)
            

    return action_space, similarities

In [16]:
def perform_action(command, p):
    p.stdin.write(bytes(command+ "\n", 'ascii'))
    p.stdin.flush()
    sleep(0.1)## wait for action to register

In [17]:
def kill_game(p):
    p.terminate()
    p.kill()

In [18]:
def choose_random_action(p, q, action_space):
    action = random.sample(action_space, 1)
    return action[0]

In [19]:
def generate_embeddings(text, model):
    text = word_tokenize(text)
    embed = model[text]
    print(embed)

In [39]:
def buildWord2VecModel():
    #model = Word2Vec.load('tutorial.model')
    f = open('tutorials_2.txt', 'r')
    tutorials = f.read()
    sentences = word_tokenize(tutorials)
    model = Word2Vec([sentences])
    return model

In [21]:
def create_LSTM():
    input_dim = 50
    embed_dim = 16
    lstm_dim = 32 
    dense_dim = 8
    

    state_input = Input(shape=(50,), name='state_input')
    action_input = Input(shape=(50,), name='action_input')

    x = concatenate([state_input, action_input])
    x = Embedding(300, embed_dim)(x)
    lstm_out = LSTM(lstm_dim)(x)

    state_out = Dense(dense_dim, activation='tanh', name='state_out')(lstm_out)
    action_out = Dense(dense_dim, activation='tanh', name='action_out')(lstm_out)

    model = Model(inputs=[state_input, action_input], outputs=[state_out, action_out])
    model.compile(loss='cosine_proximity', optimizer='adam',metrics = ['accuracy'])
    print(model.summary())
    return(model)

In [22]:
def encodeText(text):
    vocabSize = 300
    inputSize = 4
    encodedText = [one_hot(text, 100) for t in text]
    paddedText = pad_sequences(encodedText, maxlen=50, padding='post')
    return paddedText

In [23]:
encodeText('hello').shape

(5, 50)

In [65]:
encodeText('hi bye yes').shape

(10, 50)

In [40]:
model = buildWord2VecModel()

In [41]:
pbar = ProgressBar(maxval=50)
pbar.start()
p, q, t, score, moves = start_game()
moves_count = 0
story = pd.DataFrame(columns=['Surroundings', 'Inventory', 'Action', 'Response', 'Score', 'Moves'])
#lstm = create_LSTM()
try:
    for i in range(0,1):
        ## Check surroundings, check inventory, choose action, check action response
        narrative,score,moves = readLine(q)
        check_inventory(p)
        inventory,s,m = readLine(q)

        nouns = get_nouns(narrative)

        current_action_space = generate_action_tuples(nouns)


        action_space = set()
        print(action_space)
        action_space, probs = add_to_action_space(action_space, current_action_space)
        actions = []

        for a in action_space:
            actions.append(a)


        probs = np.array(probs)
        probs /= probs.sum()
        df = pd.DataFrame()
        df['Action'] = actions
        df['Prob'] = probs
        #action = choose_random_action(p,q, basic_actions)

        action = selectOne(actions, probs)


        perform_action(action, p)

        response,score,moves = readLine(q)

        story.loc[i] = [narrative, inventory, action, response, str(s), str(i+1)]

        encodedNarrative = encodeText(narrative)
        encodedAction = encodeText(action)
        print(encodedNarrative.shape, encodedAction.shape)
        #lstm.fit([encodeText(narrative), encodeText(action)], epochs=1)

        look_surroundings(p)

        pbar.update(i) #this adds a little symbol at each iteration
    pbar.finish()
    kill_game(p)
except Exception as e: 
    kill_game(p)
    print(e.with_traceback[0])

C:\Users\sparr\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  if sys.path[0] == '':
C:\Users\sparr\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).


set()


C:\Users\sparr\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
C:\Users\sparr\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
C:\Users\sparr\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  from ipykernel import kernelapp as app


(139, 50) (9, 50)


100% |########################################################################|


In [33]:
story

,Surroundings,Inventory,Action,Response,Score,Moves
0,West of House You are standing in an open...,You are empty-handed.,u,You can't go that way.,0,1
1,West of House You are standing in an open fie...,You are empty-handed.,u,You can't go that way.,0,2
2,West of House You are standing in an open fie...,You are empty-handed.,d,You can't go that way.,0,3
3,West of House You are standing in an open fie...,You are empty-handed.,u,You can't go that way.,0,4
4,West of House You are standing in an open fie...,You are empty-handed.,d,You can't go that way.,0,5


In [117]:
story['Surroundings'][0]

'    West of House  You are standing in an open field west of a white house, with a boarded  front door.  There is a small mailbox here.    '

In [36]:
df[df.Action == 'open mailbox']

,Action,Prob
120,open mailbox,1.599887e-07


In [35]:
df.Prob.mean()

0.002262443438914032

In [42]:
df.sort_values(by='Prob', ascending=False)

,Action,Prob
320,go south,3.276397e-02
354,go west,3.276096e-02
233,go north,3.270565e-02
56,go east,3.269940e-02
177,open door,3.253570e-02
378,go door,3.250474e-02
397,get door,3.229903e-02
81,go northwest,3.214676e-02
28,read door,3.190438e-02
207,wait door,3.151592e-02


In [50]:
model.save('tutorial.model')

In [52]:
model = buildWord2VecModel()

In [55]:
model.similarity('open', 'door')

C:\Users\sparr\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


0.9984921

In [109]:
kill_game(p)
